In [66]:
# Package imports
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [67]:
seed = 7
# set a seed so that the results are consistent
np.random.seed(seed)

x1 = np.random.randn(1000,1)
x1 = x1[x1>0].astype(int)
x2 = np.random.randn(1000,1)
x2 = x2[x2>0].astype(int)

n = min(x1.shape[0], x2.shape[0])

x1 = x1[0:n]
x2 = x2[0:n]
x3 = np.bitwise_xor(x1,x2)

X = np.stack((x1, x2), axis = 0).T
Y = x3.reshape((1,x3.shape[0])).T
classes = len(np.unique(Y))
inputs = X.shape[1]
print('Number of inputs = ', inputs)
print('Number of classes = ', classes)
print('X.shape = ', X.shape)
print('Y.shape = ', Y.shape)

Number of inputs =  2
Number of classes =  4
X.shape =  (485, 2)
Y.shape =  (485, 1)


In [68]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

/Users/Admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Admin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [69]:
# define model
def one_layer_NN():
    model = Sequential()
    model.add(Dense(classes, input_dim = inputs, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [70]:
estimator = KerasClassifier(build_fn=one_layer_NN, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 85.18% (7.28%)


In [71]:
# define model
def two_layer_NN():
    model = Sequential()
    model.add(Dense(8, input_dim = inputs, activation = 'relu'))
    model.add(Dense(classes, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [72]:
estimator = KerasClassifier(build_fn=two_layer_NN, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 99.18% (1.36%)
